In [ ]:
!pip install numpy==1.23.5 scipy==1.10.1


In [1]:
import os
import time
import numpy as np
import cv2
import scipy.io as sio
from pathlib import Path


In [2]:
BASE = Path("/kaggle/input/csrnet-dataset/part_B_final/part_B_final")

train_img_dir = BASE / "train_data/images"
train_gt_dir  = BASE / "train_data/ground_truth"

print("Train images:", len(os.listdir(train_img_dir)))
print("Train GT:", len(os.listdir(train_gt_dir)))


Train images: 400
Train GT: 400


In [3]:
train_image_paths = sorted(list(train_img_dir.glob("*.jpg")))

train_gt_paths = [
    train_gt_dir / f"GT_{p.stem}.mat"
    for p in train_image_paths
]

print(train_image_paths[0])
print(train_gt_paths[0])


/kaggle/input/csrnet-dataset/part_B_final/part_B_final/train_data/images/IMG_1.jpg
/kaggle/input/csrnet-dataset/part_B_final/part_B_final/train_data/ground_truth/GT_IMG_1.mat


In [4]:
def load_points(gt_path):
    mat = sio.loadmat(gt_path)
    pts = mat["image_info"][0][0][0][0][0]
    return pts


In [5]:
test_gt = train_gt_paths[0]
print("Testing:", test_gt)

pts = load_points(test_gt)
print("Shape:", pts.shape)
print(pts[:5])


Testing: /kaggle/input/csrnet-dataset/part_B_final/part_B_final/train_data/ground_truth/GT_IMG_1.mat
Shape: (234, 2)
[[ 32.67954877 753.52165403]
 [184.60695349 690.79933098]
 [344.89733461 577.8991495 ]
 [328.1713818  536.08426746]
 [329.5652112  473.36194441]]


In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

def generate_density_map(shape, points, sigma=5):
    h, w = shape
    density = np.zeros((h, w), dtype=np.float32)
    for x, y in points:
        x = min(w - 1, max(0, int(x)))
        y = min(h - 1, max(0, int(y)))
        density[y, x] += 1
    density = cv2.GaussianBlur(density, (15, 15), sigma)
    return density

class CSRNetDataset(Dataset):
    def __init__(self, img_paths, gt_paths, transform=None, sigma=5):
        self.img_paths = img_paths
        self.gt_paths = gt_paths
        self.transform = transform
        self.sigma = sigma

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        gt_path  = self.gt_paths[idx]

        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        points = load_points(gt_path)
        density = generate_density_map((h, w), points, sigma=self.sigma)

        if self.transform:
            img_t = self.transform(img)
        else:
            img_t = T.ToTensor()(img)

        dens_t = torch.from_numpy(density).unsqueeze(0)

        return img_t.float(), dens_t.float()


In [7]:
n = len(train_image_paths)
val_size = int(0.15 * n)

train_img = train_image_paths[:-val_size]
train_gt  = train_gt_paths[:-val_size]

val_img   = train_image_paths[-val_size:]
val_gt    = train_gt_paths[-val_size:]

print("Train:", len(train_img))
print("Val:", len(val_img))


Train: 340
Val: 60


In [8]:
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

train_ds = CSRNetDataset(train_img, train_gt, transform)
val_ds   = CSRNetDataset(val_img, val_gt, transform)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=4, shuffle=False)

# Test batch
imgs, dens = next(iter(train_loader))

print("Batch images:", imgs.shape)
print("Batch density:", dens.shape)


Batch images: torch.Size([4, 3, 768, 1024])
Batch density: torch.Size([4, 1, 768, 1024])


In [9]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class CSRNet(nn.Module):
    def __init__(self):
        super(CSRNet, self).__init__()

        vgg = models.vgg16_bn(weights=models.VGG16_BN_Weights.IMAGENET1K_V1)

        self.frontend = nn.Sequential(*list(vgg.features.children())[:33])

        self.backend = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, 1)
        )

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        x = F.interpolate(x, scale_factor=8, mode='bilinear', align_corners=False)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CSRNet().to(device)


Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:02<00:00, 206MB/s] 


In [10]:
import torch.optim as optim

criterion = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)


In [11]:
def validate(loader):
    model.eval()
    mae = 0
    mse = 0

    with torch.no_grad():
        for imgs, dens in loader:
            imgs = imgs.to(device)
            dens = dens.to(device)

            out = model(imgs)

            pred = out.view(out.size(0), -1).sum(dim=1).cpu()
            gt   = dens.view(dens.size(0), -1).sum(dim=1).cpu()

            mae += torch.sum(torch.abs(pred - gt)).item()
            mse += torch.sum((pred - gt)**2).item()

    n = len(loader.dataset)
    return mae/n, (mse/n)**0.5


In [12]:
import time

print("Running timing test...")

start = time.time()
imgs, dens = next(iter(train_loader))
imgs, dens = imgs.to(device), dens.to(device)

out = model(imgs)
loss = criterion(out, dens)

optimizer.zero_grad()
loss.backward()
optimizer.step()

elapsed = time.time() - start

print(f"Time for 1 batch: {elapsed:.3f} sec")
print(f"Estimated epoch time: {(elapsed*len(train_loader))/60:.2f} min")


Running timing test...
Time for 1 batch: 1.646 sec
Estimated epoch time: 2.33 min


In [13]:
best_mae = 1e9
epochs = 20

for epoch in range(1, epochs+1):
    model.train()
    total_loss = 0
    t0 = time.time()

    for imgs, dens in train_loader:
        imgs = imgs.to(device)
        dens = dens.to(device)

        out = model(imgs)
        loss = criterion(out, dens)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    val_mae, val_rmse = validate(val_loader)
    epoch_time = (time.time() - t0) / 60

    print(f"Epoch {epoch}/{epochs} | Time: {epoch_time:.2f} min | "
          f"Loss: {total_loss:.2f} | MAE: {val_mae:.2f} | RMSE: {val_rmse:.2f}")

    torch.save(model.state_dict(), f"/kaggle/working/epoch_{epoch}.pth")

    if val_mae < best_mae:
        best_mae = val_mae
        torch.save(model.state_dict(), "/kaggle/working/partB_best.pth")
        print("Saved BEST model!")


Epoch 1/20 | Time: 1.53 min | Loss: 271026.54 | MAE: 931.06 | RMSE: 1106.04
Saved BEST model!
Epoch 2/20 | Time: 1.44 min | Loss: 6022.49 | MAE: 724.06 | RMSE: 790.29
Saved BEST model!
Epoch 3/20 | Time: 1.43 min | Loss: 1748.19 | MAE: 380.40 | RMSE: 439.47
Saved BEST model!
Epoch 4/20 | Time: 1.43 min | Loss: 1275.07 | MAE: 792.22 | RMSE: 815.32
Epoch 5/20 | Time: 1.43 min | Loss: 912.77 | MAE: 175.05 | RMSE: 213.95
Saved BEST model!
Epoch 6/20 | Time: 1.43 min | Loss: 804.83 | MAE: 305.94 | RMSE: 336.03
Epoch 7/20 | Time: 1.43 min | Loss: 669.50 | MAE: 104.22 | RMSE: 132.22
Saved BEST model!
Epoch 8/20 | Time: 1.43 min | Loss: 555.41 | MAE: 212.90 | RMSE: 234.52
Epoch 9/20 | Time: 1.43 min | Loss: 566.48 | MAE: 90.63 | RMSE: 111.84
Saved BEST model!
Epoch 10/20 | Time: 1.43 min | Loss: 467.94 | MAE: 96.64 | RMSE: 120.11
Epoch 11/20 | Time: 1.43 min | Loss: 441.94 | MAE: 156.62 | RMSE: 174.28
Epoch 12/20 | Time: 1.43 min | Loss: 412.59 | MAE: 116.95 | RMSE: 131.56
Epoch 13/20 | Time: 

In [14]:
test_img_dir = BASE / "test_data/images"
test_gt_dir  = BASE / "test_data/ground_truth"

test_image_paths = sorted(list(test_img_dir.glob("*.jpg")))
test_gt_paths = [
    test_gt_dir / f"GT_{p.stem}.mat"
    for p in test_image_paths
]

print("Test images:", len(test_image_paths))
print("Test GT:", len(test_gt_paths))


Test images: 316
Test GT: 316


In [15]:
test_ds = CSRNetDataset(test_image_paths, test_gt_paths, transform)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False)


In [16]:
best_model_path = "/kaggle/working/partB_best.pth"
model.load_state_dict(torch.load(best_model_path))
model.eval()

print("Loaded best model successfully!")


Loaded best model successfully!


In [17]:
def evaluate_test(loader):
    model.eval()
    mae = 0
    mse = 0
    results = []

    with torch.no_grad():
        for i, (imgs, dens) in enumerate(loader):
            imgs = imgs.to(device)
            dens = dens.to(device)

            out = model(imgs)

            pred = out.view(-1).sum().item()
            gt   = dens.view(-1).sum().item()

            mae += abs(pred - gt)
            mse += (pred - gt)**2

            results.append((i, pred, gt))

    mae /= len(loader.dataset)
    rmse = (mse / len(loader.dataset))**0.5
    return mae, rmse, results


In [18]:
test_mae, test_rmse, test_results = evaluate_test(test_loader)

print("============== PART B TEST RESULTS ==============")
print(f"Test MAE  = {test_mae:.2f}")
print(f"Test RMSE = {test_rmse:.2f}")
print("=================================================")


============== PART B TEST RESULTS ==============
Test MAE  = 49.66
Test RMSE = 66.20


In [19]:
# ---------- Continue training safely (resume) ----------
import time, math, torch

# CONFIG — change if you want
start_epoch = 21               # next epoch number (you have finished 20)
target_epochs = 50             # desired final epoch (e.g. 50)
batch_time_estimate = None     # will be measured
quota_limit_hours = 5.0        # conservative weekly GPU quota you saw earlier (change if you know different)
safety_margin_hours = 0.5      # leave this much safety margin

# training objects assumed to exist: model, optimizer, criterion, train_loader, val_loader, device
best_mae = 1e9
# if you already have previous best_mae from run, keep it:
try:
    # try to read best_mae from saved value if present
    import os
    if os.path.exists("/kaggle/working/partB_best.pth"):
        # load model temporarily to compute val_mae if you want; otherwise keep 1e9
        pass
except Exception:
    pass

# early stopping settings (optional)
use_early_stopping = True
patience = 6   # stop if val MAE doesn't improve for this many epochs
no_improve = 0

# measure one-batch time (re-run quick batch to estimate)
imgs, dens = next(iter(train_loader))
imgs, dens = imgs.to(device), dens.to(device)
torch.cuda.synchronize() if torch.cuda.is_available() else None
t0 = time.time()
out = model(imgs)
loss = criterion(out, dens)
optimizer.zero_grad()
loss.backward()
optimizer.step()
torch.cuda.synchronize() if torch.cuda.is_available() else None
batch_seconds = time.time() - t0
batches_per_epoch = len(train_loader)
epoch_seconds = batch_seconds * batches_per_epoch
print(f"Measured: {batch_seconds:.3f}s per batch → est {epoch_seconds/60:.2f} min / epoch")

# project remaining time if we run until target_epochs
epochs_left = max(0, target_epochs - (start_epoch-1))
projected_hours = (epoch_seconds * epochs_left) / 3600.0
print(f"Projected time for {epochs_left} more epochs: {projected_hours:.2f} hours")

# check against quota
if projected_hours + safety_margin_hours > quota_limit_hours:
    print("WARNING: projected run may exceed your quota limit. Reduce target_epochs or run fewer epochs.")
    # you can choose to abort here by uncommenting:
    # raise SystemExit("Aborting to avoid exceeding quota. Reduce target_epochs.")
else:
    print("Within quota — starting training...")

# training loop resume
for epoch in range(start_epoch, target_epochs+1):
    model.train()
    total_loss = 0.0
    t_ep = time.time()
    for imgs, dens in train_loader:
        imgs = imgs.to(device)
        dens = dens.to(device)

        out = model(imgs)
        loss = criterion(out, dens)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    val_mae, val_rmse = validate(val_loader)
    epoch_time_min = (time.time() - t_ep) / 60.0

    print(f"Epoch {epoch}/{target_epochs} | Time: {epoch_time_min:.2f} min | "
          f"Loss: {total_loss:.2f} | MAE: {val_mae:.2f} | RMSE: {val_rmse:.2f}")

    # Save checkpoint
    torch.save(model.state_dict(), f"/kaggle/working/epoch_{epoch}.pth")

    # Save best
    if val_mae < best_mae:
        best_mae = val_mae
        torch.save(model.state_dict(), "/kaggle/working/partB_best.pth")
        print("➡️ Saved BEST model!")
        no_improve = 0
    else:
        no_improve += 1

    # optional early stopping
    if use_early_stopping and no_improve >= patience:
        print(f"No improvement for {patience} epochs — stopping early at epoch {epoch}.")
        break

print("Continuing training finished (or stopped). Best MAE:", best_mae)


Measured: 0.852s per batch → est 1.21 min / epoch
Projected time for 30 more epochs: 0.60 hours
Within quota — starting training...
Epoch 21/50 | Time: 1.45 min | Loss: 360.05 | MAE: 238.82 | RMSE: 246.31
➡️ Saved BEST model!
Epoch 22/50 | Time: 1.45 min | Loss: 325.40 | MAE: 170.80 | RMSE: 179.85
➡️ Saved BEST model!
Epoch 23/50 | Time: 1.45 min | Loss: 315.38 | MAE: 426.10 | RMSE: 429.14
Epoch 24/50 | Time: 1.45 min | Loss: 310.18 | MAE: 53.33 | RMSE: 67.22
➡️ Saved BEST model!
Epoch 25/50 | Time: 1.45 min | Loss: 293.27 | MAE: 166.70 | RMSE: 173.25
Epoch 26/50 | Time: 1.45 min | Loss: 303.59 | MAE: 235.32 | RMSE: 239.56
Epoch 27/50 | Time: 1.45 min | Loss: 316.82 | MAE: 567.63 | RMSE: 569.58
Epoch 28/50 | Time: 1.45 min | Loss: 293.28 | MAE: 74.04 | RMSE: 86.08
Epoch 29/50 | Time: 1.45 min | Loss: 265.15 | MAE: 139.77 | RMSE: 145.55
Epoch 30/50 | Time: 1.45 min | Loss: 262.90 | MAE: 57.95 | RMSE: 66.27
No improvement for 6 epochs — stopping early at epoch 30.
Continuing training fin

In [20]:
model.load_state_dict(torch.load("/kaggle/working/partB_best.pth"))
model.to(device)
model.eval()

test_mae, test_rmse, test_results = evaluate_test(test_loader)

print("============== PART B TEST RESULTS ==============")
print(f"Test MAE  = {test_mae:.2f}")
print(f"Test RMSE = {test_rmse:.2f}")
print("=================================================")


============== PART B TEST RESULTS ==============
Test MAE  = 53.27
Test RMSE = 66.71
